## Main

- Brute force is $O(N^3)$
    - Loop over the array once $O(N)$ to get start index
    - Loop over the array once more to get end index $O(N)$
    - For each start/end pair, sum the array by looping over all elements $O(N)$

In [15]:
input_arr=[-2,1,-3,4,-1,2,1,-5,4]
def compute_max_subarray(arr):
    '''
    O(N^3) time, O(1) space
    '''
    maxval=0
    sums=0
    for i in range(0, len(input_arr)):
        for j in range(i, len(input_arr)):
            # print('='*50)
            # print(i,j)
            # print(input_arr[i], input_arr[j])
            # print(input_arr[i:(j+1)])
            for k in input_arr[i:(j+1)]:
                sums+=k
            # print(sums)
            if sums >= maxval:
                maxval=sums
            sums=0
    return maxval

compute_max_subarray(input_arr)

6

- Better brute force is $O(N^2)$
    - Kind of "DP" solution
    - When you sum the subarray from i to j, for i to j+1, you only need to add 1 additional value instead of summing over the whole subarray again
    - To do this, maintain an array detailing the cumulative sum up to that point in the subarray ($O(N)$ memory)
    - Then do the same nested loop to get start and end index $O(N^2)$

In [19]:
input_arr=[-2,1,-3,4,-1,2,1,-5,4]
def compute_max_subarray(arr):
    '''
    O(N^2) time, O(1) space
    '''
    maxval=0
    for i in range(0, len(input_arr)):
        sums=0
        for j in range(i, len(input_arr)):
            # print('='*50)
            # print(f'{i=},{j=}')
            # print(f'{input_arr[i]=},{input_arr[j]=}')
            sums+=input_arr[j]
            if sums >= maxval:
                maxval=sums
    return maxval

compute_max_subarray(input_arr)

6

- Best solution: $O(N)$ (**Kadane's Algorithm**)
    - Insight: If a prefix has a negative value, you always want to ignore it. 
        - For example [-1,-1,3,1]
        - The subarray [-1,-1] is always going to be negative. So there is no value in maintaining it
    - Insight 2: If a suffix is negative, you should still include it. Because a negative value can be followed by an even larger positive value
        - For example [-1,-1,3,-1,4]
        - The max subarray [3,-1,4] contains a negative number, but we tolerate it because it gets us to 4

    - So the trick: 
        - Loop over the subarray once $O(N)$
        - Maintain a pointer for "array_start"
        - As long as the cumulative sum is negative, shift this pointer to after the cumulative sum (because you'll never have to add a subarray with a negative sum)
        - At each iteration, compute the cumulative sum, and check if it is larger than the max subarray sum

In [29]:
input_arr=[-1,-1,-1]
input_arr = [-2,1,-3,4,-1,2,1,-5,4]

def compute_max_subarray(arr):
    '''
    O(N) time, O(1) space
    '''
    maxval=-float('inf')
    cumsum = 0
    for i in range(0, len(arr)):
        ## Add value at current index to cumsum
        cumsum = cumsum + arr[i]
        
        ## If cumsum is greater than maxval, set maxval to cumsum
        if cumsum > maxval:
            maxval = cumsum
        
        ## If cumsum is negative, set cumsum to 0 (i.e. ignore the values up to and including this point)
        ## Else, cumsum just accumulates values
        if cumsum < 0:
            cumsum = 0

    return maxval

compute_max_subarray(input_arr)

6

## Followup

- The follow up asks us to approach this using divide and conquer
    - Note: When you see "divide and conquer", you will usually need to involve recursion in some way

- The idea here is: We recursively split the array into 2 halves, and find the maximum subarray in each half. The actual implementation will take some time to grok

- Let's start by proving that the solution works for a subarray with length 2. For clarity, we'll test it on all possible cases;
    - A: `[1,-1]`: Max value in left subarray
    - B: `[-1,1]`: Max value in right subarray
    - C: `[1,1]`: Max value is sum of both subarrays

    - Let's start using Case A as an example
        - Let's initialise a left pointer $l=0$, a right pointer $r=1$, and a midpoint at $m = (0+1) // 2 = 0$
        - Going backwards from position $m$ to position $l$, we perform a cumulative sum and record the maximum value. In this case, it is 1
        - Going forward from position $m$ to position $r$, we perform a cumulative sum and record the maximum value. In this case, it is -1
        - Finally, we check the max cumulative sum across the entire array. In this case, it is 0
        - Finally, we take a max of the left, right and entire array, and find that it is `max(1, -1, 0) = 1`, which is exactly the max
    - The same logic will hold for cases B and C

    - $\therefore$ It is proven that this logic works for the case of array size 2

- Let's try with array length 3
    - A: `[1,-1,-2]`: Max value in left subarray of length 1
    - B: `[1,2,-1]`: Max value in left subarray of length 2
    - C: `[1,2,3]`: Max value in entire subarray
    - D: `[-1,1,2]`: Max value in right subarray of length 2
    - E: `[-1,-2,1]`: Max value in right subarray of length 1

    - Let's start using Case A as an example
        - Init left pointer $l=0$, right pointer $r=2$, mid pointer at $m = (0+2) // 2 = 1$
        - Find the `maxSubArraySum` between l=0 and m-1=0. In this case, it is just $[1]$
        - Find the `maxSubArraySum` between m+1=2 and r=2. In this case, it is just $[-2]$
        - Finally, find the `maxCrossingSum` for the entire array where $l=0$, $m=1$, and $h=2$
            - Decrement indices from m to 0, and take cumulative sum 
                - Keep a record of the largest cumulative sum `left_sum`
                - In this case, we will get `-1`, then `-1+1 = 0`. So `left_sum = 0`
            - Increment indices from m to r, and take cumulative sum 
                - Keep a record of the largest cumulative sum `right_sum`
                - In this case, we will get `-1`, then `-1+-2 = -3`. So `right_sum = -1`
            - Take the maximum of the array's total sum (which is just `left_sum + right_sum - arr[m]`), `left_sum`, and `right_sum`
            - In this case, it is `max(-2, 1, -2) = 1`
    - Amazingly, the logic for `maxCrossingSum` holds across cases A to E!
        - Case B
            - `left_sum = 3`
            - `right_sum = 2`
            - `array_sum = 2`
            - $\therefore$ `maxCrossingSum = max(3,2,2) = 3`
            - $\therefore$ `overall_max = max(maxCrossingSum, leftSubArray=1, rightSubArray=-1) = 3`
        - etc.

- Let's try with array length 4
    - A: `[1,-1,-2,-3]`: Max value in left subarray of length 1
    - B: `[-1,1,2,-2]`: Max value in middle subarray of length 2

    - Let's start using Case A as an example
        - Init left pointer $l=0$, right pointer $r=3$, mid pointer at $m = (0+3) // 2 = 1$
        - Find the `maxSubArraySum` between l=0 and m-1=0. $[1]$
        - Find the `maxSubArraySum` between m+1=2 and r=3. This is the case of subarray with length 2, and we have proven that it works. In this case, $[-2]$
        - Finally, find the `maxCrossingSum` for the entire array where $l=0$, $m=1$, and $h=3$
            - `left_sum = 0`
            - `right_sum = -6`
            - Take the maximum of the array's total sum (which is just `left_sum + right_sum - arr[m]`), `left_sum`, and `right_sum`
            - In this case, it is `max(-5, 0, -6) = 0`
        - So overall, the maximum of this array is `max(1, -2, 0) = 1`
        - To recap: 
            - the left `maxSubArraySum` checks the 1st element 
            - the right `maxSubArraySum` checks the 3rd, 4th, and 3rd+4th element (because this is just the length 2 array case discussed above)
            - the `maxCrossingSum` checks for `arr[0:2]` and `arr[1:4]`
                - If max subarray is at index 1, or index 0+1 `arr[0:2]` will return it
                - If max subarray is at index [1,2,3], [1,2] or [2,3], `arr[1:4]` will return it
            - So overall, all possibilities are checked

- For anything higher than this, basically it recursively reduces to these base cases listed

- Time complexity:
    - You recursively split the array into 2 halves, so you incur a $O(log N)$ time complexity from this step
    - For each split, you need to sum over the subarray to perform the cross sum, incurring $O(N)$ time 
    - Hence, you end up with $O(N log n)$ time complexity

- Space complexity:
    - Recursion takes up stack space. So in this case, stack will have $O(\log N)$ height
    - No other data structure is needed

In [77]:
def max_subarray_cross_sum(array, left, mid, right, nest=0, verbose=False):
    if verbose:
        print(f"{' '*nest*5} {'='*50}")
        print(f"{' '*nest*5} Entering max_subarray_cross_sum")
        print(f"{' '*nest*5} {left=} {mid=} {right=}")
    left_sum = -float('inf')
    right_sum = -float('inf')
    
    _tmp = 0
    for left_index in range(mid, left-1, -1):
        _tmp += array[left_index]
        if _tmp > left_sum:
            left_sum = _tmp
    
    _tmp = 0
    for right_index in range(mid, right+1):
        _tmp += array[right_index]
        if _tmp > right_sum:
            right_sum = _tmp
    if verbose:
        print(f"{' '*nest*5} {left_sum=} {right_sum=} {left_sum+right_sum-array[mid]=}")
        print(f"{' '*nest*5} Exiting max_subarray_cross_sum")
        print(f"{' '*nest*5} {'='*50}")
    return max(left_sum, right_sum, left_sum+right_sum-array[mid])
        

def max_subarray_sum(array, left, right, nest=0, verbose=False):
    if verbose:
        print(f"{' '*nest*5} {'-'*50}")
        print(f"{' '*nest*5} Entering max_subarray_sum")
        print(f"{' '*nest*5} {left=} {right=}")
    if left > right:
        return -float('inf')
    
    if left == right:
        return array[left]
    
    mid = (left+right) // 2
    if verbose:
        print(f"{' '*nest*5} {mid=}")

    leftsum = max_subarray_sum(array, left, mid-1, nest+1, verbose=verbose)
    rightsum = max_subarray_sum(array, mid+1, right, nest+1,verbose=verbose)
    crosssum = max_subarray_cross_sum(array, left, mid, right, nest+1, verbose=verbose)

    if verbose:
        print(f"{' '*nest*5} {leftsum=} {rightsum=} {crosssum=}")
        print(f"{' '*nest*5} Exiting max_subarray_sum")
        print(f"{' '*nest*5} {'-'*50}")

    return max(leftsum,rightsum,crosssum)

input_arr=[-2,1,-3,4,-1,2,1,-5,4]
max_subarray_sum(input_arr, 0, len(input_arr)-1, verbose=True)

 --------------------------------------------------
 Entering max_subarray_sum
 left=0 right=8
 mid=4
      --------------------------------------------------
      Entering max_subarray_sum
      left=0 right=3
      mid=1
           --------------------------------------------------
           Entering max_subarray_sum
           left=0 right=0
           --------------------------------------------------
           Entering max_subarray_sum
           left=2 right=3
           mid=2
                --------------------------------------------------
                Entering max_subarray_sum
                left=2 right=1
                --------------------------------------------------
                Entering max_subarray_sum
                left=3 right=3
                Entering max_subarray_cross_sum
                left=2 mid=2 right=3
                left_sum=-3 right_sum=1 left_sum+right_sum-array[mid]=1
                Exiting max_subarray_cross_sum
           leftsum=-inf r

6